In [1]:
import os
os.environ["API_HOST"]=""
os.environ["CONSUMER_KEY"] = ""
os.environ["CONSUMER_SECRET"]=""

In [2]:
from an_common.resource import get_resource_doc, get_building_metadata
from an_common.api import get_app_data
from an_common.day_type import cal_day_type
import pandas as pd
import json
from DATA.CODE_REFERENCES.ENVIRONMENT.an_common_env import get_nantum_data, calc_daily_peak_demand, calc_ghg_emission
from datetime import datetime

# always use datetime to generate the date of relative days
today = datetime.today().strftime('%Y-%m-%d')


In [3]:
## To get a list of company names active in Nantum
company_list = get_app_data(route='companies')
# the full list of company can be found as first-level keys in get_nantum_data_dict

In [5]:
## TO get a list of building names given a company name for example 'jpmc'
company = 'jpmc'
bldg_list = get_resource_doc(company, None, resource=None, query=None, extract_all=True, use_beta=None, domain='apps/self/namespaces')
# the full list of buildings can be found as second-level keys in get_nantum_data_dict

In [5]:
## To get a dictionary of building metadata using 'jpmc' for company and 'tempe_north_building_f' for building as an example
company = 'jpmc'
building = 'tempe_north_building_f'
bldg_meta_data = get_building_metadata(company, building)
# You can find more explanation about the output 'bldg_meta_data' below

the "bldg_meta_data" output is a dictionary, there is a list of these keys you can query for:['_id',
 'access_management_provider',
 'address',
 'base_year',
 'billing_resources',
 'carbon_factor_source_id',
 'company',
 'coordinates',
 'dis',
 'floor_area',
 'floor_area_units',
 'floors',
 'img',
 'interior_space_comfort',
 'key_zone_identifier',
 'logo',
 'nantum_start_date',
 'owner',
 'startup_sla_time',
 'timezone',
 'type',
 'ven_id',
 'pricing_plan_id',
 'pinned',
 'rank',
 'building_address',
 'status',
 'currency',
 'pricing_plan']. The most commonly used ones include "floor_area" and "floor_area_units" for the floor area and unit, normally in sqft, for the building, "building_address" to get the address info of the building, "timezone" for the timezone string for the building.

In [15]:
## To get specific building metadata explained above, such as "timezone", you can modify the get_building_metadata function by adding the specific key
building = '767_fifth_ave'
company = 'boston_properties'
tz = get_building_metadata(company, building, 'timezone')
print(tz)

America/New_York


In [16]:
## To get a list of all available sensor resources in a building, resource is needed to be used to get nantum data
company = 'rudin'
building = '345_Park'
sensors = get_resource_doc(company, building, resource='sensors', query=None, extract_all=True, use_beta=None, domain='buildings')
sensor_resource_list = list(pd.DataFrame(sensors)['resource'].unique())
print(sensor_resource_list)
# the full list of resource can be found as get_nantum_data_dict under "sensor_resources"

['air_co2', 'air_flow', 'air_humidity', 'air_quality_indices', 'analytics_comfort_indices', 'building_mode', 'chilled-water_demand', 'chilled_water_temperature', 'co2_emission_rates', 'co2_emissions', 'co2_level', 'condenser-water_demand', 'condenser_water_temperature', 'constants', 'cooling_load', 'current', 'electric_consumption', 'electric_consumption_clean', 'electric_consumption_fan_vfd', 'electric_demand', 'electric_demand_clean', 'electric_demand_vfd', 'energy_consumption', 'energy_star_occupancy', 'energy_star_score', 'energy_star_site_eui', 'energy_star_source_eui', 'firecom', 'firecom_events', 'frequency', 'interior_space_temperature', 'kpis', 'miscellaneous', 'miscellaneous_temp', 'nantum_enable', 'nantum_signal', 'occupancy', 'occupancy_clean', 'occupancy_density', 'occupancy_flow', 'occupancy_flow_indices', 'outside_air_co2_level', 'outside_air_temperature', 'perimeter_space_temperature', 'power', 'pump_speed', 'pump_status', 'return_air_temperature', 'return_fan_speed', '

In [17]:
## Get a pandas Series of the day types, True if the date is working day, False if it's not.
company = 'rudin'
building = '345_Park'
start_date = '2024-06-05'
end_date = '2024-06-06'
day_type_out = cal_day_type(company, building, None, start_date, end_date, use_beta=None)
print(day_type_out)
# You can look at each day by using day_type_out['2024-06-05'] as an example

2024-06-05    True
2024-06-06    True
Freq: D, dtype: bool


In [20]:
## To get sensor metadata
company = 'jpmc'
building = 'tempe_north_building_f'
sensors = get_resource_doc(company, building, resource='sensors',
                 query=None, extract_all=True, use_beta=None, domain='buildings')
# You can find more explanation about the output 'sensors' below

the "sensors" output is a list of dictionaries for each sensor. For each sensor, there is a list of these keys you can query for:
['_id',
 'benchmark',
 'building',
 'cid_a1',
 'cid_a2',
 'cid_a3',
 'cid_s1',
 'clone_id',
 'color',
 'company',
 'cov',
 'cov_inc',
 'cumulative',
 'description',
 'dis',
 'equipment_id',
 'floor',
 'hidden',
 'identifier1',
 'identifier2',
 'identifier3',
 'identifier4',
 'identifier5',
 'kind',
 'last_value',
 'location',
 'max_val',
 'min_val',
 'model_number',
 'multiplier',
 'name',
 'prediction',
 'relationships',
 'reset',
 'resource',
 'source',
 'status',
 'tags',
 'total',
 'units',
 'user_tags',
 'last_time']
 The most commonly used ones include "_id" for the unique sensor_id for each sensor, "units" to get the unit of the sensor, "dis" for the display name of the sensor

In [24]:
## Use get_nantum_data to load total electricity data in the unit of kW
company = 'jpmc'
building = '383_madison'
start_date = '2024-05-01'
end_date = '2024-05-01'
resource = 'electric_demand'
# note that the granularity is the time interval of the data, for example '60min','15min', etc.
# By default we use '60min' because when ask for a day's total, we can sum up the data for each hour directly
# but if the granularity is set to '15min', we need to divide the data by 4 (60/15) to get the total for each day
gran='60min'
## Notice that the returned nantum data includes both start and end date
# total='only' means that only the total sensor is used
df_elec_demand = get_nantum_data(company, building, resource, start_date, end_date, gran=gran, total='only')
## to calculate one day's energy consumption
## when gran='60min', sum directly; if gran='15min', then divide by 4
df_elec_total = df_elec_demand.sum() 
company = 'rudin'
building = '345_Park'
start_date = '2024-05-01'
end_date = '2024-05-01'
resource = 'interior_space_temperature'
gran='60min'
df_T_int = get_nantum_data(company, building, resource, start_date, end_date, gran=gran, rename=True)
## Use get_nantum_data to get weather data
company = 'rudin'
building = '345_Park'
start_date = '2024-05-01'
end_date = '2024-05-01'
resource = 'weather_history'
gran='60min'
df_weather = get_nantum_data(company, building, resource, start_date, end_date, gran=gran)

In [5]:
## an example to calculate ghg data
company = 'rudin'
building = '345_Park'
start_date = '2024-05-01'
end_date = '2024-05-01'
df_ghg = calc_ghg_emission(company, building, start_date, end_date, gran='60min',
                      elec_ghg_factor=None, steam_ghg_factor=None, gas_ghg_factor=None)
df_ghg

,elec_ghg_kgCO2,steam_ghg_kgCO2,total_ghg_kgCO2
time,,,
2024-05-01 00:00:00,603.353241,64.375704,667.728945
2024-05-01 01:00:00,582.547956,60.799276,643.347232
2024-05-01 02:00:00,577.924560,60.799276,638.723836
2024-05-01 03:00:00,566.366069,60.799276,627.165345
2024-05-01 04:00:00,564.054371,75.104988,639.159359
2024-05-01 05:00:00,568.677767,53.646420,622.324187
2024-05-01 06:00:00,663.457395,75.104988,738.562383
2024-05-01 07:00:00,1463.304986,1573.628320,3036.933306
2024-05-01 08:00:00,1289.927618,1734.567580,3024.495198


In [6]:
company = 'rudin'
building = '345_Park'
start_date = '2024-05-01'
end_date = '2024-05-01'
df_peak_demand = calc_daily_peak_demand(company, building, start_date, end_date)
df_peak_demand

,peak_demand
time,
2024-05-01,4989.0


In [8]:
df_occ = get_nantum_data(company, building, 'occupancy', start_date, end_date, gran='15min', total='only')
df_occ
## Note that there are three columns for the occupancy data, representing the in and out occupancy, and the total occupancy (in minus out), respectively.
## The in and out occupancy are the number of people entering and leaving the building, respectively. 
## Please use integers for occupancy.
## Do not use column names as they may change via buildings.

,56e32daef3b2c8dfb7069d4a|in,56e32daef3b2c8dfb7069d4a|out,56e32daef3b2c8dfb7069d4a
time,,,
2024-05-01 00:00:00,1.333333,6.000000,0.000000
2024-05-01 00:15:00,3.666667,29.333333,0.000000
2024-05-01 00:30:00,4.000000,60.333333,0.000000
2024-05-01 00:45:00,4.000000,70.666667,0.000000
2024-05-01 01:00:00,4.000000,75.000000,0.000000
...,...,...,...
2024-05-01 22:45:00,8724.333333,8634.000000,90.333333
2024-05-01 23:00:00,8728.666667,8654.333333,74.333333
2024-05-01 23:15:00,8731.666667,8669.000000,62.666667
